In [34]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq, Trainer, TrainingArguments
)
import torch



In [35]:
# 🔹 1️⃣ Load model + tokenizer
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# 🔹 2️⃣ Move model to the right device (MPS or CPU)
device = "mps" if torch.backends.mps.is_available() else "cpu"
model.to(device)
print(f"✅ Using device: {device}")

# 🔹 3️⃣ Load dataset
dataset = load_dataset("json", data_files={
    "train": "FinalData/train.jsonl",
    "validation": "FinalData/val.jsonl"
})

✅ Using device: mps


Generating train split: 33567 examples [00:00, 393992.88 examples/s]
Generating validation split: 3730 examples [00:00, 416715.60 examples/s]


In [36]:

# 🔹 4️⃣ Preprocess function
def preprocess_function(batch):
    inputs = tokenizer(batch["input_text"], truncation=True, padding="max_length", max_length=128)
    targets = tokenizer(batch["target_text"], truncation=True, padding="max_length", max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized = dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Map: 100%|██████████| 3730/3730 [00:00<00:00, 17501.78 examples/s]


In [45]:
from transformers import TrainingArguments
import torch

# ✅ Device setup (MPS if available, else CPU)
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✅ Using Apple Metal (MPS) for acceleration")
else:
    device = torch.device("cpu")
    print("⚠️ Using CPU, training will be slower")

model.to(device)
import os
os.environ["ACCELERATE_DISABLE_MPS_FALLBACK"] = "1"
os.environ["ACCELERATE_USE_CPU"] = "false"

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./results_flan_t5_base_mac",
    eval_strategy="no",       # 👈 disable evaluation
    learning_rate=3e-4,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=False,                     # ❌ disable fp16 on Mac
    bf16=False,
    logging_dir="./logs",
    logging_steps=100,
    save_strategy="epoch",          # save checkpoint after each epoch
    save_total_limit=2,
    load_best_model_at_end=False,   # no eval, so we skip best model tracking
    report_to="none"                # disable wandb/tensorboard
)

✅ Using Apple Metal (MPS) for acceleration


In [46]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)


/var/folders/vd/mw3l_3gd1yl7xf_86l9lrnk80000gn/T/ipykernel_96959/1926513511.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [47]:
# 🔹 7️⃣ Start training
trainer.train()

/Users/rahul/Desktop/Arnav/FormulaVerseModel/venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
100,2.938100
200,0.999600
300,0.941000
400,0.851600
500,0.893400
600,0.818500
700,0.799000
800,0.804200
900,0.794500
1000,0.786800


/Users/rahul/Desktop/Arnav/FormulaVerseModel/venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/rahul/Desktop/Arnav/FormulaVerseModel/venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=50352, training_loss=0.3594158888620765, metrics={'train_runtime': 37735.605, 'train_samples_per_second': 2.669, 'train_steps_per_second': 1.334, 'total_flos': 1.7238937537216512e+16, 'train_loss': 0.3594158888620765, 'epoch': 3.0})

In [48]:

# 🔹 8️⃣ Save the fine-tuned model
trainer.save_model("./flan_t5_base_formulaVerse_mac")
print("✅ Model fine-tuned and saved to ./flan_t5_base_formulaVerse_mac")

✅ Model fine-tuned and saved to ./flan_t5_base_formulaVerse_mac


In [22]:


# 🔹 3️⃣ Example input (change this to match your dataset format)
input_text = "Template: LINEAR:add(ARG0,ARG1)|multiply(CONST_4,CONST_4)|multiply(ARG1,CONST_100)|multiply(TEMP1,CONST_1000)|multiply(ARG3,TEMP0)|add(TEMP3,TEMP2)|add(TEMP5,TEMP4)|divide(TEMP6,ARG3). Arguments: ARG0: 80, ARG1: 4, ARG3: 16. Task: Generate a realistic math word problem that matches this equation."

# 🔹 4️⃣ Tokenize input
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# 🔹 5️⃣ Generate output
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_length=128,
        num_beams=4,
        temperature=0.7,
        early_stopping=True
    )

# 🔹 6️⃣ Decode and print
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n🧠 Model Output:", generated_text)



🧠 Model Output: a , b and c enter into partnership . a invests 80 % of the capital and b invests 4 % of the capital and 16 % of the capital respectively . in what ratio the profit earned after 2 years be divided between a and b respectively ?


In [56]:
from simple_parser import expression_to_template
def generate_word_problem(equation: str, max_length: int = 128):
    # Step 1: Convert equation to template format
    parsed = expression_to_template(equation)
    if not parsed["success"]:
        print(f"❌ Parser Error: {parsed['error']}")
        return None

    input_text = parsed["formatted_input"]
    print("\n🧩 Parsed Input to Model:\n", input_text)

    # Step 2: Tokenize
    inputs = tokenizer(
        input_text, return_tensors="pt", truncation=True, padding=True
    ).to(device)

    # Step 3: Generate output
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=0.9,
            num_beams=4,
            repetition_penalty=1.2,
            early_stopping=True,
        )

    # Step 4: Decode result
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("\n🧮 Generated Word Problem:\n", result)
    return result


In [ ]:
import sys
!{sys.executable} -m pip install -U google-genai


In [5]:
from handler import generate_output

In [6]:

# Test different equations
cases = [
    "20 + 4 + 5",
    "x + y = 5",
    "a + b + c = 12",
    "2x + 3y = 33",
    "23*66/11"
]

for eq in cases:
    print(f"\nEquation: {eq}")
    print("Generated Problem:", generate_output(eq))



Equation: 20 + 4 + 5
Generated Problem: Maria is setting up a bake sale booth. She already has 20 cookies. Her mom brings her 4 more cookies, and her neighbor drops off 5 cupcakes. How many total baked goods does Maria have for the bake sale?

Equation: x + y = 5
Generated Problem: Okay, here's a word problem for the equation x + y = 5:

"Maria is making a friendship bracelet with beads. She wants to use a total of 5 beads. If she uses 'x' number of blue beads and 'y' number of red beads, how many different combinations of blue and red beads can she use to make the bracelet?"

Equation: a + b + c = 12
Generated Problem: Okay, here's a word problem:

"Maria is making a fruit salad. She wants to use 12 pieces of fruit in total. If she uses 5 strawberries and 4 blueberries, how many grapes does she need to add to the salad?"

Equation: 2x + 3y = 33
Generated Problem: Maria is selling lemonade and cookies at her school fair. She sells lemonade for $2 a cup and cookies for $3 each. At the 